In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.icd10data.com/ICD10CM/Codes"
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
response = requests.get(url,headers=headers)

In [ ]:
file_name = f'{url}.html'.replace('https://www.', '').replace('/', '_')

with open(file_name, 'w', encoding='utf-8') as file:
    file.write(response.text)

In [ ]:
with open(file_name, 'r', encoding='utf-8') as f:
    html_content = f.read()
soup = BeautifulSoup(html_content, 'lxml')
body_content = soup.find('div', class_ = "body-content")


<div class="body-content">
 <h1 class="pageHeading">
  2025 ICD-10-CM Codes
 </h1>
 <ul>
  <li>
   <a class="identifier" href="/ICD10CM/Codes/A00-B99">
    A00-B99
   </a>
   <div class="tip images-note" data-textdivname="x1" data-titledivname="z1">
   </div>
   Certain infectious and parasitic diseases
  </li>
  <li>
   <a class="identifier" href="/ICD10CM/Codes/C00-D49">
    C00-D49
   </a>
   <div class="tip images-note" data-textdivname="x24" data-titledivname="z24">
   </div>
   Neoplasms
  </li>
  <li>
   <a class="identifier" href="/ICD10CM/Codes/D50-D89">
    D50-D89
   </a>
   <div class="tip images-note" data-textdivname="x46" data-titledivname="z46">
   </div>
   Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism
  </li>
  <li>
   <a class="identifier" href="/ICD10CM/Codes/E00-E89">
    E00-E89
   </a>
   <div class="tip images-note" data-textdivname="x54" data-titledivname="z54">
   </div>
   Endocrine, nutritional and metabo

In [16]:
links = body_content.find_all('a')

In [19]:
for link in links:
    href = link.get('href')
    if href and href.startswith('/ICD10CM/Codes'):
        full_url = f"https://www.icd10data.com{href}"
        with open('links.text', 'a', encoding='utf-8') as f:
            f.write(full_url + '\n')

In [20]:
with open('links.text', 'r', encoding='utf-8') as f:
    all_links = f.readlines()
    all_links = [link.strip() for link in all_links]

print(all_links)

['https://www.icd10data.com/ICD10CM/Codes/A00-B99/A00-A09', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A15-A19', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A20-A28', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A30-A49', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A50-A64', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A65-A69', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A70-A74', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A75-A79', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A80-A89', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/A90-A99', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B00-B09', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B10-B10', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B15-B19', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B20-B20', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B25-B34', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B35-B49', 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B50-B6

In [21]:
new_url = all_links[0]
print(new_url)

https://www.icd10data.com/ICD10CM/Codes/A00-B99/A00-A09
